# Backtest Analysis Notebook

This notebook is for backtesting trading strategies and analyzing performance.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Load data
# data = pd.read_csv("../data/btc_usdt.csv")
# print(f"Data shape: {data.shape}")
# data.head()